# Anything2Img 测试

本 notebook 用于测试 `src/autocoder/common/anything2img.py` 模块的功能。

In [1]:
# 导入必要的模块
import os
import byzerllm
from autocoder.common import AutoCoderArgs
from autocoder.common.anything2img import Anything2Img
from loguru import logger

# 初始化测试环境
logger.remove()
logger.add(lambda msg: print(msg), level="INFO")

# 创建测试输出目录
test_output_dir = "./test_output"
os.makedirs(test_output_dir, exist_ok=True)

# 初始化 AutoCoderArgs
args = AutoCoderArgs(
    output=test_output_dir,
    model="deepseek_chat",
    vl_model="qwen_vl"
)

# 初始化 LLM
llm = byzerllm.ByzerLLM.from_default_model(model="deepseek_chat")
vl_llm = byzerllm.ByzerLLM.from_default_model(model="qwen_vl")
llm.setup_sub_client("vl_model", vl_llm)

# 初始化 Anything2Img
converter = Anything2Img(llm, args)

2025-01-04 15:18:19,876	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2025-01-04 15:18:19,924	INFO worker.py:1740 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


2025-01-04 15:18:19.802 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...



2025-01-04 15:18:20,119	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2025-01-04 15:18:20,120	INFO worker.py:1582 -- Calling ray.init() again after it has already been called.


2025-01-04 15:18:20.063 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...



## 测试 PDF 转换

In [2]:
from autocoder.common.anything2img import Page
# 测试 PDF 文件转换
pdf_file = "/Users/allwefantasy/Downloads/002-auto-coder.chat-初级入门指南.pdf"  # 请确保测试目录下有这个文件
if os.path.exists(pdf_file):
    image_paths = converter.convert_pdf(pdf_file)
    print(f"转换后的图片路径: {image_paths}")
    
    # 测试图片分析
    for img_path in [image_paths[0]]:
        page_info = converter.analyze_image.with_llm(vl_llm).with_return_type(Page).run(img_path)
        print(f"图片分析结果: {page_info}")
else:
    print(f"测试文件 {pdf_file} 不存在，跳过 PDF 测试")

转换后的图片路径: ['./test_output/002-auto-coder.chat-初级入门指南.pdf_page1.png', './test_output/002-auto-coder.chat-初级入门指南.pdf_page2.png', './test_output/002-auto-coder.chat-初级入门指南.pdf_page3.png', './test_output/002-auto-coder.chat-初级入门指南.pdf_page4.png', './test_output/002-auto-coder.chat-初级入门指南.pdf_page5.png', './test_output/002-auto-coder.chat-初级入门指南.pdf_page6.png', './test_output/002-auto-coder.chat-初级入门指南.pdf_page7.png', './test_output/002-auto-coder.chat-初级入门指南.pdf_page8.png', './test_output/002-auto-coder.chat-初级入门指南.pdf_page9.png', './test_output/002-auto-coder.chat-初级入门指南.pdf_page10.png', './test_output/002-auto-coder.chat-初级入门指南.pdf_page11.png', './test_output/002-auto-coder.chat-初级入门指南.pdf_page12.png', './test_output/002-auto-coder.chat-初级入门指南.pdf_page13.png', './test_output/002-auto-coder.chat-初级入门指南.pdf_page14.png']
图片分析结果: text='002-auto-coder.chat 初级入门指南\n我们在上一篇介绍了如何安装 auto-coder.chat 回 Windows: auto-coder.chat 安装指南\n MacOS/Linux 直接 pip install -U auto-coder 即可。测试过的 python 版本为：3.10/3

## 测试 DOCX 转换

In [ ]:
# 测试 DOCX 文件转换
docx_file = "test.docx"  # 请确保测试目录下有这个文件
if os.path.exists(docx_file):
    image_paths = converter.convert_docx(docx_file)
    print(f"转换后的图片路径: {image_paths}")
    
    # 测试图片分析
    for img_path in image_paths:
        page_info = converter.analyze_image.with_llm(llm).with_return_type(Page).run(img_path)
        print(f"图片分析结果: {page_info}")
else:
    print(f"测试文件 {docx_file} 不存在，跳过 DOCX 测试")

## 测试 Markdown 生成

In [5]:
# 测试 Markdown 生成
pdf_file = "/Users/allwefantasy/Downloads/002-auto-coder.chat-初级入门指南.pdf"
markdown_content = converter.to_markdown(pdf_file)
print("生成的 Markdown 内容:")
print(markdown_content)

# 保存 Markdown 文件
output_md = os.path.join(test_output_dir, "output.md")
with open(output_md, "w", encoding="utf-8") as f:
    f.write(markdown_content)
print(f"Markdown 文件已保存到: {output_md}")

2025-01-04 15:20:44.273 | INFO     | autocoder.common.anything2img:to_markdown:135 - Analyzed ./test_output/002-auto-coder.chat-初级入门指南.pdf_page1.png

2025-01-04 15:20:52.293 | INFO     | autocoder.common.anything2img:to_markdown:135 - Analyzed ./test_output/002-auto-coder.chat-初级入门指南.pdf_page2.png



IndexError: page 1 not in document

In [4]:
import json
json.loads("""
{
    "text": "002-auto-coder.chat 初级入门指南\n\n我们在上一篇介绍了如何安装 auto-coder.chat 回Windows: auto-coder.chat 安装指南\n\n MacOS/Linux 直接 pip install -U auto-coder 即可。测试过的 python 版本为： 3.10/3.11\n\n初始化设置\n\n在 PowerShell/CMD 或者 VSCode 内置的 terminal 中输入如下指令（auto-coder.chat）：\n\nPS C: \\Users\\Administrator\\projects\\auto-coder> auto-coder.chat\n\n记得不要忘记了使用 conda activate auto-coder 切换到合适的环境虚拟环境哟。\n\n第一次运行，系统会自动要求你提供一个大模型供应商（你后续可以改，比如用本地的模型或者其他第三方模型），此时会弹出一个对话框：",
    "images": [
        {
            "coordinates": [0.05, 0.39, 0.95, 0.59],
            "text": "对图片的描述"
        }
    ],
    "width": 页面宽度,
    "height": 页面高度
}
""")

JSONDecodeError: Invalid control character at: line 3 column 40 (char 42)

## 清理测试环境

In [ ]:
# 删除测试输出目录
import shutil
shutil.rmtree(test_output_dir)
print(f"已删除测试输出目录: {test_output_dir}")